In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [5]'.</span>

An Exception was encountered at 'In [5]'.

In [1]:
#This cell is tagged parameters for running this template using Papermill to inject values to overwrite these.
markdown = ""
section = 99
sub_pg = 99
end_nb = False
next_nb = ""


In [2]:
# Parameters
markdown = "\n# Part 2 of Helper Tools: Reordering Ideograms to Minimize Overlap\n\nThis page is primarily based on the following page at the Circos documentation site:\n\n\n\n- [2. Reordering Ideograms to Minimize Overlap](????????????)\n\n\n\nThat page is found as part number 4 of the ??? part ['Helper Tools' section](http://circos.ca/documentation/tutorials/quick_start/) of [the larger set of Circos tutorials](http://circos.ca/documentation/tutorials/).\n\n\n\nGo back to Part 1 by clicking [here &#8592;](HelperTools_Part1.ipynb).\n\n\n\n----\n\n\n\n9 --- Helper Tools\n==================\n\n2. Reordering Ideograms to Minimize Overlap\n-------------------------------------------\n\n::: {#menu4}\n[[Lesson](/documentation/tutorials/utilities/minimize_crossing/lesson){.clean}]{.active}\n[Images](/documentation/tutorials/utilities/minimize_crossing/images){.normal}\n:::\n\nscript location\n---------------\n\n    tools/orderchr\n\nscript usage\n------------\n\n    > cd tools/orderchr\n    > ./run\n    ...\n    improvement(minimize) orig 1164 final 124 change 89.35%\n    chromosomes_order = chr9,chr5,chr15,chr16,chr18,chr14,chr3,chr20,chr7,chr8,chr10,chr4,chr12,chr6,chr2,chr17,chr1,chr19,chr13,chr11\n\nThe final line reported by the script provides the suggested\nchromosomes\\_order - copy this line into the circos.conf file.\n\nTo get the full manpage, use -man.\n\n    > cd tools/orderchr\n    > bin/orderchr -man\n\nAdjust the configuration file etc/orderchr.conf to suit your needs.\n\nYou can generate link data randomly using the randomlinks tool.\n\ndetails\n-------\n\nThe script minimizes (or maximizes) the link cross score, which is a\nquantity that estimates the number of links that cross in the image. The\nlink score is influenced solely by the start and end position\n(chromosome and coordinate) of each link. The score is an estimate\nbecause the link geometry (Bezier curve settings) are not taken into\naccount. The link score does not include any crossing between\nintra-chromosomal links, since no amount of chromosome shuffling can\nmitigate these crossings.\n\nThe script is currently not capable of working with chromosome ranges\nand axis breaks. In other words, you cannot optimize layout of\nchromosomes which have been broken into components (e.g. chr2-\\>\nchr2a:0-10Mb, chr2b:20-30Mb, chr2c:30-40Mb, etc). If you want to use\naxis breaks, you\\'ll have to create new chromosomes (e.g., chr2a, chr2b,\nchr2c, \\...) that represent the components of the original chromosome\n(chr2) and adjust the karyotype and link data file accordingly.\n\nGiven a set of links L, sorted by angular position of link start, the\nlink cross score is given by\n\n![](/circos/images/eq-crossing.png)\n\nwhere the terms are as follows. Each link pair (i,j) (angular position\nof start of link i is greater than that of j) contributes +1 to the\ncrossing score if the links cross. Whether they cross is determined by\nthe position of the angular positions of the links\\' start/end\npositions. The links are deemed to cross if the angular positions are\ninterleaved.\n\n![](/circos/images/circos-cross.png)\n\n### deterministic optimization\n\nOne of the ways in which Circos\\'s links can be effectively used to\nvisualize alignment data between two related structures. These\nstructures could be two genomes, or two similar regions of the same\ngenome, or even two physically different representations of the same\nstructure (e.g. a fingerprint map and sequence assembly).\n\n![](/circos/images/circos-cross-02.png)\n\nWhen a Circos is used to show alignments, it is very helpful to have the\nchromosomes ordered in such a way as to reduce link crossing.\nChromosomes should be ordered so that any link crossing is an indication\nof real differences in the underlying structures.\n\nFor example, consider [this image](#img1) which shows a comparison\nbetween two sets of structures, arbitrarily named chromosomes a1,a2,\\...\nand b1,b2,\\... It is obvious that most of the data relate one of the\n\\\"b\\\" structures to many \\\"a\\\" structures. For example, these type of\ndata sets are common when comparing a single contiguous structure (e.g.\nchromsome sequence assembly, supercontig, etc) to many smaller\nstructures (e.g., resequencing contigs). However, because the\nchromosomes a\\* and b\\* are not well ordered, a lot of the links cross\nand it is not possible to determine whether the crossing is due to the\norder or to irreducible (i.e., leading to crossings that cannot be\nmitigated by rearrangement) differences in the chromosomes.\n\nThis kind of link relationships can be deconvoluted by using the\ndeterministic optimization component of the orderchr script. This\noptimization reorders chromosomes based on degree of connectivity. Given\none chromosome, all chromosomes connected to it are placed immediately\nafter it, in decreasing order of the number of shared links. The [result\nof this shuffling](#img2) quickly produces a view of the data that is\nless cluttered.\n\nWhen applying the deterministic optimization, it is helpful to place all\nthe chromosomes from one set first (e.g. all the b\\* chromosomes) to\nstraighten the links ([example](#img2)). Even though more links cross in\nthe [right panel of this example](#img2), it is easier to spot\nblock-level differences. Initial order is controlled using\n-init\\_order\\* parameters.\n\n![](/circos/images/circos-cross-03.png)\n\nThis optimization is termed \\\"deterministic\\\" because the algorithm\nalways produces the same layout, and in part, to contrast the other type\nof optimization, which is stochastic (read below). The order created by\nthe deterministic optimization is suitable as input to the stochastic\noptimization routine, and for this reason the deterministic ordering is\nrefered to as \\\"warmup\\\" in the documentation and configuration of\norderchr.\n\n### stochastic optimization\n\nThe stochastic routine uses simulated annealing (SA) to derive an\nchromosome layout that minimizes (or maximizes) the link score function.\nThe best solution is not guaranteed, however, and I encourage you to run\nthe simulation several times to determine the convergence of the\nsolution. Ideally, all instances of the simulation should return layouts\nwhose score is very similar (e.g. with in 1-2%).\n\nThe stochastic routine can be initialized with the order from the\ndeterministic optimization, or initialized with an arbitrary order. The\n-init\\_order\\* parameters control the order. Furthermore, certain\nchromosomes can be forced to remain static during the shuffling using\n-static\\* parameters. This is helpful when you wish to keep the order of\ncertain structures fixed.\n\nIn [this example image](#img4), I show the same link data with six\ndifferent chromosome layouts. The link score function is shown in the\ncenter of the image.\n\nDepending on the nature of the data, you may not be able to\nsignificantly reduce link crossing. When the majority of your\nchromosomes are highly inter-connected, then any order results in a lot\nof crossings. The link crossing score in the [above example](#img4)\ncould be reduced from 835 to 262, a 70% improvement. However, in this\n[segmental duplication data set](#img5), the score cannot be reduced by\nmore than about 50%.\n\norderchr manpage\n----------------\n\n-   [NAME](#NAME)\n-   [SYNOPSIS](#SYNOPSIS)\n-   [DESCRIPTION](#DESCRIPTION)\n-   [DEFINING WHICH CHROMOSOMES TO\n    SHUFFLE](#DEFINING_WHICH_CHROMOSOMES_TO_SH)\n    -   [MODE 1 - shuffle set defined by link\n        data](#MODE_1_shuffle_set_defined_by_)\n    -   [MODE 2 - shuffle set defined by\n        file](#MODE_2_shuffle_set_defined_by_)\n    -   [MODE 3 - shuffle set defined by regular\n        expression](#MODE_3_shuffle_set_defined_by_)\n    -   [MODE 4 - shuffle set defined by\n        list](#MODE_4_shuffle_set_defined_by_)\n    -   [MODE 5 - multimode](#MODE_5_multimode)\n-   [DEFINING INITIAL ORDER](#DEFINING_INITIAL_ORDER)\n-   [DEFINING WHICH CHROMOSOMES TO REMAIN\n    ANCHORED](#DEFINING_WHICH_CHROMOSOMES_TO_RE)\n-   [CONTROLING THE OPTIMIZATION](#CONTROLING_THE_OPTIMIZATION)\n    -   [Warmup Round](#Warmup_Round)\n    -   [Stochastic Optimization Round](#Stochastic_Optimization_Round)\n-   [SIMULATED ANNEALING](#SIMULATED_ANNEALING)\n    -   [iterations](#iterations)\n    -   [max\\_flips, min\\_flips](#max_flips_min_flips)\n    -   [temp0](#temp0)\n    -   [optimize = minimize\\|maximize](#optimize_minimize_maximize)\n-   [SIMULATION SCHEDULE](#SIMULATION_SCHEDULE)\n    -   [No Warmup, Single Stochastic\n        Round](#No_Warmup_Single_Stochastic_Rou)\n    -   [No Warmup, Multiple Stochastic\n        Rounds](#No_Warmup_Multiple_Stochastic_R)\n    -   [Warmup, Multiple Stochastic\n        Rounds](#Warmup_Multiple_Stochastic_Roun)\n-   [HISTORY](#HISTORY)\n-   [BUGS](#BUGS)\n-   [AUTHOR](#AUTHOR)\n-   [CONTACT](#CONTACT)\n\n------------------------------------------------------------------------\n\n------------------------------------------------------------------------\n\n[NAME]{#NAME}\n=============\n\norderchr - determine a chromosome order to minimize cross-over of links\n\n------------------------------------------------------------------------\n\n[SYNOPSIS]{#SYNOPSIS}\n=====================\n\n      orderchr -links linkfile.txt \n               -karyotype karyotype.txt \n               { -shuffle_file chrs_to_shuffle.txt | -shuffle LIST | -shuffle_rx REGEX_LIST } \n               {-static LIST} {-static_rx REGEX_LIST}\n               {-init_order LIST} {-init_order_rx REGEX_LIST}\n\n------------------------------------------------------------------------\n\n[DESCRIPTION]{#DESCRIPTION}\n===========================\n\nBy examining the frequencies of chromosome-chromosome relationships\ndefined in the link file, this script suggests a new order for\nchromosomes that results in fewer cross-overs between links. Simulated\nannealing is used to optimize the chromosome order (read below about\nparameters). Run the simulation a couple of times to check convergence\n(finding a global minimum is not guaranteed).\n\n------------------------------------------------------------------------\n\n[DEFINING WHICH CHROMOSOMES TO SHUFFLE]{#DEFINING_WHICH_CHROMOSOMES_TO_SH}\n==========================================================================\n\nThe set of chromosomes to shuffle is specified by either (a) link data,\nwhereby all chromosomes that have links are subject to shuffling (b)\n-shuffle\\_file, whereby only those chromosomes listed in the file are\nshuffled (c) -shuffle, whereby the set of chromosomes to shuffle is\nspecified by a list, and (d) -shuffle\\_rx, whereby only those\nchromosomes that match a regular expression are shuffled. Any chromosome\nthat is not identified by one of these methods does not participate in\nthe shuffling process. Specifically, any links to/from such chromosomes\nare not considered when minimizing link crossing.\n\n------------------------------------------------------------------------\n\n[MODE 1 - shuffle set defined by link data]{#MODE_1_shuffle_set_defined_by_}\n----------------------------------------------------------------------------\n\n      > orderchr -links linkfile.txt\n\nAll chromosomes mentioned in the -links file will be subject to\nreordering. The initial order will be taken from order of appearance in\nthe karyotype file.\n\n------------------------------------------------------------------------\n\n[MODE 2 - shuffle set defined by file]{#MODE_2_shuffle_set_defined_by_}\n-----------------------------------------------------------------------\n\n      > orderchr -links linkfile.txt -shuffle_file chrs_to_shuffle.txt\n\nThe set of chromosomes to shuffle is given in the -shuffle\\_file file,\nwhich contains one chromosome per line. For example\n\n      > cat chrs_to_shuffle.txt\n      chr1\n      chr5\n      chr12\n      chr17\n      ...\n\nThe initial order will be taken from order of appearance in the file.\n\n------------------------------------------------------------------------\n\n[MODE 3 - shuffle set defined by regular expression]{#MODE_3_shuffle_set_defined_by_}\n-------------------------------------------------------------------------------------\n\n      > orderchr -links linkfile.txt -shuffle_rx chr1\n\nSame as MODE 1, except that chromosome list will be filtered using the\nregular expression and only those chromosomes that match the regular\nexpression are shuffled.\n\nIn this example, chromosomes matching \\`\\`chr1\\'\\' will be shuffled\n(e.g. chr1, chr10, chr11, etc).\n\n------------------------------------------------------------------------\n\n[MODE 4 - shuffle set defined by list]{#MODE_4_shuffle_set_defined_by_}\n-----------------------------------------------------------------------\n\n      > orderchr -links linkfile.txt -shuffle chr1,chr2,chr6,chr7,chr10\n\nSame as MODE 3, except that chromosomes are specified by a list.\n\nIn this example, chromosomes chr1,chr2,chr6,chr7, and chr10 will be\nshuffled.\n\n------------------------------------------------------------------------\n\n[MODE 5 - multimode]{#MODE_5_multimode}\n---------------------------------------\n\nYou can combine -shuffle\\_file, -shuffle\\_rx and -shuffle to additively\ndefine the shuffle list.\n\n------------------------------------------------------------------------\n\n[DEFINING INITIAL ORDER]{#DEFINING_INITIAL_ORDER}\n=================================================\n\nThe initial order of chromosomes can be defined in two ways. First, the\nmethod that is used to specify which chromosomes to shuffle will dictate\nthe initial order. Modes 1 and 2 (see above) use the order of\nchromosomes as they appear in the karyotype file. Mode 3 (see above)\nuses the order from the shuffle file.\n\nYou can override the initial order using the -init\\_order parameter. The\nvalue of this parameter is expected to a comma-delimited list of\nchromosomes, which may be the full set or a subset of chromosomes.\n\nFor example, if the entire set of chromosomes to shuffle is chr1..chr5,\nthen you can specify the initial order which explicitly orders each\nchromosome\n\n      -init_order chr2,chr5,chr1,chr3,chr4\n\nor just a subset\n\n      -init_order chr2,chr5\n\nIn the latter case, the final order will be\n\n      { chr2,chr5 } , { chr1,chr3,chr4 }\n\ncomprised of two order groups: leading group of chromosomes as ordered\nby -init\\_order and a group of remaining chromosomes, in order of\nappearance as set by parameters in the section DEFINING WHICH\nCHROMOSOMES TO SHUFFLE.\n\nIf a chromosome mentioned in -init\\_order is not a candidate for\nshuffling, its mention in the order string will be ignored.\n\nThe option -init\\_order\\_rx works just like -init\\_order, except that\nthe list a list of regular expressions rather than chromsome names. For\nexample,\n\n      -init_order chr1,chr2\n\nis equivalent to\n\n      -init_order { chrs matching /chr1/ },{ chrs matching /chr2/ }\n\nand for the canonical human genome with standard order this would be\n\n      -init_order chr1,chr10,chr11,chr2,chr20,chr21,chr22\n\nSince this is a subset of chromosomes, the final initial order will be\nautomatically completed by chromosomes from the karyotype file that were\nnot explicitly ordered\n\n      chr1, chr10, chr11, chr2, chr20, chr21, chr22, chr3..chr9, chr12..chr19, chrX, chrY\n\nIf both -init\\_order\\_rx and -init\\_order are defined, order is\ninitially defined by -init\\_order\\_rx and then refined using\n-init\\_order. Thus\n\n      -init_order chr10,chr20,chrx -init_order_rx chr1,chr2\n\nwill result in\n\n      chr10, chr20, chrx, chr1, chr11, chr2, chr21, chr22, chr3..chr9, chr12..chr19 ,chrY\n\n------------------------------------------------------------------------\n\n[DEFINING WHICH CHROMOSOMES TO REMAIN ANCHORED]{#DEFINING_WHICH_CHROMOSOMES_TO_RE}\n==================================================================================\n\nAfter the set of chromosomes to shuffle has been defined, and the\ninitial order has been set, you can define a subset of chromosomes to\nremain in the same order (static) throughout the shuffling process.\n\nThe difference between a chromosome (a) not being part of a shuffle set\nand (b) being part of a shuffle set, but remain static, is that in the\nformer, links to chromosomes do not play a role in the ordering process\nwhereas in the latter case links to these chromosomes contribute to the\nshuffle score. Thus, chromosomes which are static have all non-static\nchromosomes shuffled around them in order to minimize link crossover.\n\nDefining static chromosomes is done by a comma-delimited list of regular\nexpressions\n\n      > orderchr -links linkfile.txt -static_rx chr1\n\nIn this example, all chromosomes matching the regular expression chr1\nwill not have their order adjusted. Any links to/from these chromosomes\nwill contribute to the total link crossing score, but the chromosomes\nthemselves will not be moved. For example, if the original order of\nchromosomes is\n\n      chr1,chr2,chr3,chr10,chr11,chr20,chr21\n\nthen any shuffle solution will have the order\n\n      chr1,-,-,chr10,chr11,-,-\n\nwith chr1, chr10 and chr11 remaining fixed.\n\nTo define multiple regular expressions, use a list of regular\nexpressions.\n\n      > orderchr -links linkfile.txt -static_rx chr1,x,y\n\nLike with -init\\_order, you can use the chromosome names to define\nstatic entries using -static.\n\n      -static chr1,chr2,chr3\n\nwill keep chromosomes chr1, chr2 and chr3 always in the same position.\nYou can combine -static\\_rx and -static\n\n      -static_rx chr1,chr2 -static chrx,chry\n\nin which case all chromosomes that match either the regular expressions\ndefined by -static\\_rx or the names defined by -static will be kept in\nthe same position during shuffling.\n\n------------------------------------------------------------------------\n\n[CONTROLING THE OPTIMIZATION]{#CONTROLING_THE_OPTIMIZATION}\n===========================================================\n\nThe order optimization process comprises one or more rounds. Each round\nis defined by a \\<round\\> block in the \\<simulation\\> block\n\n      <simulation>\n       <round>\n        # settings for round 1\n       </round>\n       <round>\n        # settings for round 2\n       </round>\n       ...\n      </simulation>\n\nA round can be either a warmup (read below), or a full simulated\nannealing process (read below). The outcome of the warmup is\ndeterministic, and thus the warmup should only be used as the first\n(optional) round.\n\n------------------------------------------------------------------------\n\n[Warmup Round]{#Warmup_Round}\n-----------------------------\n\nDuring the warmup round, the initial order of the chromosomes is defined\nbased on the degree of connectivity between chromosome pairs.\n\nThis warmup is most suited for data sets in which most relationships\nbetween chromosomes are many-to-one (e.g., chromosome A has links to\nmany chromosomes B,C,D,\\... but each of B,C,D generally only links to\nA). Many-to-one data sets are common for alignments (e.g. chr A\ncorresponds to the chromosome whereas and points to ctg A, B, C, \\...\nall sequence contigs that map to disjoint regions on chr A).\n\nThe warmup algorithm is as follows. The chromosome (chrA) with the most\nlinks is selected first and used to initialize the new order. A list of\nall links to chrA is created, grouped by chromosome, and sorted based on\nthe average position of the link on chrA. Chromosomes are added to the\nnew order based on descending order of grouped link position. Once all\nchromosomes are placed, the next unplaced chromosome with the most links\nis selected and the process continues until all chromosomes are placed.\n\nThe warmup is deterministic - it will result in the same order each\ntime. It is insensitive to the initial order, or values of -static and\n-static\\_rx.\n\n      <round>\n       warmup = yes\n      </round>\n\n------------------------------------------------------------------------\n\n[Stochastic Optimization Round]{#Stochastic_Optimization_Round}\n---------------------------------------------------------------\n\nAfter the optional warmup round, all other rounds should be of\nstochastic type (this is the default round type, if warmup=yes is not\nset).\n\nParameters for the round are defined as follows\n\n      <round>\n        iterations = 1000\n        max_flips  = 10\n        min_flips  = 2\n        temp0      = 0.01\n      <round>\n\nFor the details of each parameter, read the section below. You can set\nparameter values to be relative to values of the previous round by\nprefixing the parameter with \\`\\`r\\'\\'. For example,\n\n      <round>\n        iterations = 1000\n        ...\n        temp0      = 0.01\n      <round>\n\n      <round>\n        iterations = r2\n        ...\n        temp0      = r0.5\n      <round>\n\n      <round>\n        iterations = r2\n        ...\n        temp0      = r0.5\n      <round>\n\ndefines three rounds. The first round has 1000 iterations with\ntemp0=0.01. The second round has 2x iterations (2000) and a value of\ntemp0 of 0.5\\*0.01=0.005. The third round has again 2x iterations (4000)\nand temp0 of 0.5\\*0.005=0.0025.\n\nRelative parameter values are very useful for additional rounds when the\ntransition probability is decreased (temp0 is lowered). You can decrease\ntemp0 in relative steps, without needing to remember what the previous\nvalue was. This allows you to create a multi-round optimization schedule\nwith all parameter defined in a single place (first round).\n\nThe solution at the end of a round is used as the initial order for the\nnext round.\n\n------------------------------------------------------------------------\n\n[SIMULATED ANNEALING]{#SIMULATED_ANNEALING}\n===========================================\n\nThis method is an optimization method that encourages the discovery of a\nglobal minimum by traversing the space of solutions with a small (and\ndecreasing as simulation runs) chance of visiting less desirable\nsolutions.\n\nThere are three parameters that control the optimization.\n\n------------------------------------------------------------------------\n\n[iterations]{#iterations}\n-------------------------\n\nThe number of iterations to perform. At each iteration, the current\nsolution is randomly modified and either accepted or rejected.\n\n------------------------------------------------------------------------\n\n[max\\_flips, min\\_flips]{#max_flips_min_flips}\n----------------------------------------------\n\nThe optimization run is split into max\\_flips-min\\_flips+1 equal-sized\nintervals. During each iterval, the number of random chromosome pair\nswaps in the solution is given by\n\n      min_flips + (max_flips-min_flips)*(1-t)\n\nwhere t is a relative round completion time t=0..1 at the current\niteration.\n\nFor example, if max\\_flips is 5 and min\\_flips is 2 and iterations=1000.\nThen the number of random pair swaps is\n\n      iteration 1-249    5\n      iteration 250-499  4\n      iteration 500-749  3\n      iteration 750-1000 2\n\nI suggest starting with a value that corresponds to 5% of the\nchromosomes. For example, if you have 100 chromosomes, use max\\_flips=5\nto start. It\\'s also a good idea to set min\\_flips=1 for the last round\nto avoid abandoning the solution (remember that in simulated annealing\nit is possible to discard a solution for a worse solution).\n\n------------------------------------------------------------------------\n\n[temp0]{#temp0}\n---------------\n\nThis parameter determines the probability of a transition to a less\ndesirable solution. The transition probability is\n\n      p(dE) = temp0*exp( - dE/t )\n\nwhere t=1..0 over the length of the simulation and dE is the relative\nchange in the desirability of two solutions.\n\nIf temp0=1, then the probability of accepting a solution that is 10%\nworse (e.g. dE=0.1) is\n\n      p(0.1) = exp (-0.1/1)   = 90%    at start of simulation\n             = exp (-0.1/0.5) = 82%    half way through simulation\n             = exp (-0.1/0.1) = 37%    90% of the way through simulation\n\nBy lowering temp0, you lower the probability of transition to a less\ndesirable solution.\n\nDo not adjust temp0 unless you feel that the simulation is (a) not\ntraversing the solution space sufficiently - in which case make temp0\nlarger or (b) too many low-quality solutions are accepted - in which\ncase make temp0 smaller.\n\n------------------------------------------------------------------------\n\n[optimize = minimize\\|maximize]{#optimize_minimize_maximize}\n------------------------------------------------------------\n\nMost of the time you\\'ll want to adjust the chromosome order in a way to\nminimize the number of crossing links. However, you can set to maximize\nthe number of crossing links by setting\n\n      optimize = maximize\n\n------------------------------------------------------------------------\n\n[SIMULATION SCHEDULE]{#SIMULATION_SCHEDULE}\n===========================================\n\n------------------------------------------------------------------------\n\n[No Warmup, Single Stochastic Round]{#No_Warmup_Single_Stochastic_Rou}\n----------------------------------------------------------------------\n\n      <simulation>\n      <round>\n       iterations = 1000\n       max_flips  = 10 # or set this to ~5% of your chromosomes\n       min_flips  = 1\n       temp0      = 0.01\n      </round>\n      </simulation>\n\n------------------------------------------------------------------------\n\n[No Warmup, Multiple Stochastic Rounds]{#No_Warmup_Multiple_Stochastic_R}\n-------------------------------------------------------------------------\n\nThe purpose of rounds 2 and 3 is to successively decrease the transition\nprobability to worse solutions and also decrease the degree to which\nsuccessive candidate solutions vary from the current solution. In these\nrounds, a more careful search is carried out around the solution\nprovided in round 1.\n\n      <simulation>\n      <round>\n       iterations = 1000\n       max_flips  = 10 # or set this to ~5% of your chromosomes\n       min_flips  = 1\n       temp0      = 0.01\n      </round>\n      </simulation>\n      <simulation>\n      <round>\n       iterations = r2    # 2000\n       max_flips  = 2\n       min_flips  = 1\n       temp0      = r0.5  # 0.005\n      </round>\n      <round>\n       iterations = r2    # 4000\n       max_flips  = 1\n       min_flips  = 1\n       temp0      = r0.1 # 0.0005\n      </round>\n      </simulation>\n\n------------------------------------------------------------------------\n\n[Warmup, Multiple Stochastic Rounds]{#Warmup_Multiple_Stochastic_Roun}\n----------------------------------------------------------------------\n\n      <simulation>\n      <round>\n       warmup = yes\n      </round>\n      <round>\n       iterations = 1000\n       max_flips  = 10 # or set this to ~5% of your chromosomes\n       min_flips  = 1\n       temp0      = 0.01\n      </round>\n      </simulation>\n      <simulation>\n      <round>\n       iterations = r2    # 2000\n       max_flips  = 2\n       min_flips  = 1\n       temp0      = r0.5  # 0.005\n      </round>\n      <round>\n       iterations = r2    # 4000\n       max_flips  = 1\n       min_flips  = 1\n       temp0      = r0.1 # 0.0005\n      </round>\n      </simulation>\n\n------------------------------------------------------------------------\n\n[HISTORY]{#HISTORY}\n===================\n\n-   **[14 July 2008]{#item_14}**\n    Expanded documentation and added \\_rx parameters.\n-   **[8 July 2008]{#item_8}**\n\n    Started and versioned.\n\n------------------------------------------------------------------------\n\n[BUGS]{#BUGS}\n=============\n\n------------------------------------------------------------------------\n\n[AUTHOR]{#AUTHOR}\n=================\n\nMartin Krzywinski\n\n------------------------------------------------------------------------\n\n[CONTACT]{#CONTACT}\n===================\n\n      Martin Krzywinski\n      Genome Sciences Centre\n      Vancouver BC Canada\n      www.bcgsc.ca\n      martink@bcgsc.ca\n\n"
section = 9
sub_pg = 2
end_nb = False
next_nb = "HelperTools_Part3.ipynb"


In [3]:
'''from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
printmd(markdown)'''
print(markdown) #use this because want to be able to edit the markdown and rendering it with `printmd` diables that


# Part 2 of Helper Tools: Reordering Ideograms to Minimize Overlap

This page is primarily based on the following page at the Circos documentation site:



- [2. Reordering Ideograms to Minimize Overlap](????????????)



That page is found as part number 4 of the ??? part ['Helper Tools' section](http://circos.ca/documentation/tutorials/quick_start/) of [the larger set of Circos tutorials](http://circos.ca/documentation/tutorials/).



Go back to Part 1 by clicking [here &#8592;](HelperTools_Part1.ipynb).



----



9 --- Helper Tools

2. Reordering Ideograms to Minimize Overlap
-------------------------------------------

::: {#menu4}
[[Lesson](/documentation/tutorials/utilities/minimize_crossing/lesson){.clean}]{.active}
[Images](/documentation/tutorials/utilities/minimize_crossing/images){.normal}
:::

script location
---------------

    tools/orderchr

script usage
------------

    > cd tools/orderchr
    > ./run
    ...
    improvement(minimize) orig 1164 final 124 change 89.35



----

### Generating the plot produced by this example code

The following two cells will generate the plot. The first cell adjusts the current working directory.

In [4]:
%cd ../circos-tutorials-0.67/tutorials/{section}/{sub_pg}/

/home/jovyan/circos-tutorials-0.67/tutorials/9/2


In [5]:
%%bash
../../../../circos-0.69-6/bin/circos -conf circos.conf

debuggroup summary 0.21s welcome to circos v0.69-6 31 July 2017 on Perl 5.022000
debuggroup summary 0.22s current working directory /home/jovyan/circos-tutorials-0.67/tutorials/9/2
debuggroup summary 0.22s command ../../../../circos-0.69-6/bin/circos -conf circos.conf
debuggroup summary 0.22s loading configuration from file circos.conf

  *** CIRCOS ERROR ***

      cwd: /home/jovyan/circos-tutorials-0.67/tutorials/9/2

      command: ../../../../circos-0.69-6/bin/circos -conf circos.conf

  CONFIGURATION FILE ERROR

  Circos could not find the configuration file [circos.conf]. To run Circos, you
  need to specify this file using the -conf flag. The configuration file
  contains all the parameters that define the image, including input files,
  image size, formatting, etc.

  If you do not use the -conf flag, Circos will attempt to look for a file
  circos.conf in several reasonable places such as . etc/ ../etc

  To see where Circos looks for the file, use

      circos -debug_flag io

Use of uninitialized value in subroutine entry at /home/jovyan/circos-0.69-6/bin/../lib/Circos/Configuration.pm line 781.
 at /home/jovyan/circos-0.69-6/bin/../lib/Circos/Error.pm line 425.
	Circos::Error::fatal_error("configuration", "missing", "circos.conf") called at /home/jovyan/circos-0.69-6/bin/../lib/Circos/Configuration.pm line 796
	Circos::Configuration::loadconfiguration("circos.conf") called at /home/jovyan/circos-0.69-6/bin/../lib/Circos.pm line 148
	Circos::run("Circos", "_argv", "-conf circos.conf", "configfile", "circos.conf", "_cwd", "/home/jovyan/circos-tutorials-0.67/tutorials/9/2") called at ../../../../circos-0.69-6/bin/circos line 536


CalledProcessError: Command 'b'../../../../circos-0.69-6/bin/circos -conf circos.conf\n'' returned non-zero exit status 255.

View the plot in this page using the following cell.

In [ ]:
from IPython.display import Image
Image("circos.png")

----

In [ ]:
next_md = ("Continue on to the next part of the Quick Start series by clicking [here &#10145;]({})</font>.".format(next_nb))
if end_nb:
    next_md = ("End of tutorials.")
#printmd(next_md)
print(next_md) #use this because want to be able to edit the markdown and rendering it with `printmd` diables that

----